### Divided import statements

### SimulationWrapper

In [ ]:
import os
import logging
import argparse
import importlib
import h5py
import multiprocessing as mp
import numpy as np
import pandas as pd

from RateMatrix_Calcs import basic_calcs


def _workers_count():
    cpu_count = 1
    try:
        cpu_count = len(os.sched_getaffinity(0))
    except AttributeError:
        cpu_count = os.cpu_count()
    return cpu_count


def wrapper(row):
    statesDictPath = os.path.join(outputPath, 'StatesDict.npy')
    set_formatted = 'set_{:05}'.format(row[0])

    # Calculating
    RateMatrix, Dimensions, StatesDict = modelFunc.main(row)
    eigenValues, probVec, prob2D, timeScales = basic_calcs(RateMatrix, Dimensions)

    sim_results = (outputPath, set_formatted, RateMatrix, Dimensions, eigenValues, probVec, prob2D, timeScales)

    if not os.path.isfile(statesDictPath):
        np.save(os.path.join(outputPath, 'StatesDict.npy'), StatesDict)
    return sim_results


def output_handler(sim_results):
    h5_save_path = os.path.join(sim_results[0][0], 'simulation_data.h5')
    h5_file = h5py.File(h5_save_path, "a")

    for sim_result in sim_results:
        # Breaking down Rate Matrix to allow for HDF5 saving
        # See https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html for reconstruction info
#        import scipy.io as sio
#        sio.savemat('ratematrix_py.mat', {'RateMatrix':sim_result[2]})
        g = h5_file.create_group('RateMatrix/' + sim_result[1])
        g.create_dataset('data', data=sim_result[2].data, compression='gzip')
        g.create_dataset('indptr', data=sim_result[2].indptr, compression='gzip')
        g.create_dataset('indices', data=sim_result[2].indices, compression='gzip')
        g.attrs['shape'] = sim_result[2].shape
        g.attrs['dimensions'] = sim_result[3]

        # Saving other outputs
        h5_file.create_dataset('EigenValues/' + sim_result[1], data=sim_result[4])
        h5_file.create_dataset('ProbVec/' + sim_result[1], data=sim_result[5])
        h5_file.create_dataset('Prob2D/' + sim_result[1], data=sim_result[6])
        h5_file.create_dataset('TimeScales/' + sim_result[1], data=sim_result[7])
    h5_file.close()
    logging.info('Finished '+sim_results[-1][1])


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument("-p", "--parameter", dest="parameterFile",
                        help="Parameter csv file that contains the parameters for the simulations.", required=True)
    parser.add_argument("-o", "--output", dest="outputPath",
                        help="Folder path to output simulation results to.", required=True)
    parser.add_argument("-m", "--model", dest="modelFile",
                        help="Model file used to calculate the RateMatrix, found in models/ .", required=True)
    parser.add_argument("-pe", "--parallel", dest="runParallel", action='store_true',
                        help="When -pe called, simulations will attempt to be run in parallel")

    # Load inputs
    args = parser.parse_args()
    modelFile = "models." + args.modelFile
    parametersDF = pd.read_csv(args.parameterFile, index_col=0)
    outputPath = os.path.abspath(args.outputPath)
    parametersDF['OutputPath'] = outputPath

    # Setting up logger
    outputLogger = logging.getLogger()
    outputLogger.setLevel(logging.DEBUG)
    logOutputFile = os.path.join(outputPath, 'simulations.log')
    formatter = logging.Formatter('%(message)s')
    fh = logging.FileHandler(logOutputFile, 'w')
    fh.setLevel(logging.INFO)
    fh.setFormatter(formatter)
    outputLogger.addHandler(fh)

    # Number of simulations each worker will run
    chunk_size = 4

    global modelFunc
    modelFunc = importlib.import_module(modelFile)

    # Creating blank hdf5 file
    savePathHDF5 = os.path.join(outputPath, 'simulation_data.h5')
    f = h5py.File(savePathHDF5, "w")
    f.attrs['model_name'] = modelFunc.model_name()
    f.close()

    if args.runParallel:
        # Getting worker count and initiating pool
        num_workers = _workers_count()
        print('Starting pool with', num_workers, 'workers')
        pool = mp.Pool(processes=num_workers)

        # Distributing tasks and executing
        for i in range(0, len(parametersDF), chunk_size):
            slc = parametersDF.iloc[i: i+chunk_size]
            pool.map_async(wrapper, slc.itertuples(name=None), callback=output_handler)
        pool.close()
        pool.join()

    else:
        print('Running simulations sequentially')
        for i in range(0, len(parametersDF), chunk_size):
            slc = parametersDF.iloc[i: i+chunk_size]
            sequential_sim_results = [wrapper(row) for row in slc.itertuples(name=None)]
            output_handler(sequential_sim_results)


### RateMatrix_Calcs

In [ ]:
import numpy as np
from scipy.sparse.linalg import eigs

def basic_calcs(matrix, dimensions, dims_to_reduce=(2,3), nsig=15, sigma=1e-12):
    dim = matrix.shape[0]
    eigen_values, right_eigen_vectors = eigs(matrix, k=nsig, sigma=sigma, v0=np.ones(dim))

    prob_vec = np.real(right_eigen_vectors[:,0])/np.real(np.sum(right_eigen_vectors[:,0]))
    prob_full_d = prob_vec.reshape(dimensions, order='F')
    prob_2d = np.sum(prob_full_d, axis=dims_to_reduce)

    time_scales = -np.divide(1, eigen_values[1:].real)

    return eigen_values, prob_vec, prob_2d, time_scales

### common_calcs

In [ ]:
from numba.decorators import jit

@jit('Tuple((f8[:], f8[:], f8[:]))(f8[:], f8[:], f8[:], i8[:], i8, i8, f8[:], i8[:,:], i8, i8)')
def Update_RateMatrix(CurInd_vals, DestInd_vals, RateMatrix_vals, Cur, CurInd, DestInd, parameters, laws, m, n):
    par = parameters[m]
    law = laws[m,:]
    for i in range(Cur.shape[0]):
        tmp1 = Cur[i]**law[i]
        tmp2 = 1
        for j in range(law[i]):
            tmp2 = tmp2 * (j+1)
        par = par * (tmp1/tmp2)

    CurInd_vals[n] = CurInd
    DestInd_vals[n] = DestInd
    RateMatrix_vals[n] = par

    return CurInd_vals, DestInd_vals, RateMatrix_vals


### Compute_RateMatrix_MISAChromatin

In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
from .common_calcs import Update_RateMatrix


def model_name():
    return 'MISAChromatin'

def MISAChromatin_Rxn(parameters):
    # Parameters and model name
    model = 'MISAChromatin'
    N = parameters[1]
    kd = parameters[2]
    g0_a = parameters[3]
    g1_a = parameters[4]
    g2_a = parameters[5]
    g3_a = parameters[6]
    g0_b = parameters[7]
    g1_b = parameters[8]
    g2_b = parameters[9]
    g3_b = parameters[10]
    ha = parameters[11]
    hr = parameters[12]
    fa = parameters[13]
    fr = parameters[14]
    c_c = parameters[15] # chromatin closing
    c_cr = parameters[16] # chromatin opening
    c_o = parameters[17] # chromatin closing with repressor bound
    model_name = model + '_N{}'.format(N)

    # Copy number lists
    A = list(range(N+1))
    B = list(range(N+1))

    # Defining Gene States (manually)
    GeneA_00, GeneA_01, GeneA_10, GeneA_11, GeneA_c = [0,1], [0,1], [0,1], [0,1], [0,1]
    GeneB_00, GeneB_01, GeneB_10, GeneB_11, GeneB_c = [0,1], [0,1], [0,1], [0,1], [0,1]
    GeneA_States=[[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]]
    GeneB_States=[[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]]

    # Number of Microstate = CopyA * CopyB * NumGeneStatesA * NumGeneStatesB
    NumStates = (N+1) * (N+1) * len(GeneA_States) * len(GeneB_States)

    # Initializing Rxn dict to hold Reactions, Species and Parameters
    # Currently NumRxn and NumSpec need to be set manually
    Rxn = {}
    NumRxn = 36
    NumSpec = 12

    Rxn['Parameters'] = np.array([g0_a,g1_a,g2_a,g3_a,ha,hr,hr,ha,fa,fr,fa,fr,kd, # Gene A reaction rates/parameters
                                  g0_b,g1_b,g2_b,g3_b,ha,hr,hr,ha,fa,fr,fa,fr,kd, # Gene B reaction rates/parameters
                                  c_c,c_c,c_cr,c_c,c_o,                           # Gene A chromatin rates/parameters
                                  c_c,c_c,c_cr,c_c,c_o])                          # Gene B chromatin rates/parameters
    #for i in Rxn['Parameters']:
    #    print(i)
    #print('')
    Rxn['Law'] = np.zeros((NumRxn, NumSpec), dtype=int)
    Rxn['Stoich'] = np.zeros((NumRxn, NumSpec), dtype=int)

    # Reaction Rate Laws, number of each species involved in the reaction
    Rxn['Law'][0,2]=1
    Rxn['Law'][1,3]=1
    Rxn['Law'][2,4]=1
    Rxn['Law'][3,5]=1
    Rxn['Law'][4,0]=2
    Rxn['Law'][4,2]=1
    Rxn['Law'][5,1]=2
    Rxn['Law'][5,2]=1
    Rxn['Law'][6,1]=2
    Rxn['Law'][6,3]=1
    Rxn['Law'][7,0]=2
    Rxn['Law'][7,4]=1
    Rxn['Law'][8,3]=1
    Rxn['Law'][9,4]=1
    Rxn['Law'][10,5]=1
    Rxn['Law'][11,5]=1
    Rxn['Law'][12,0]=1

    Rxn['Law'][13,7]=1
    Rxn['Law'][14,8]=1
    Rxn['Law'][15,9]=1
    Rxn['Law'][16,10]=1
    Rxn['Law'][17,1]=2
    Rxn['Law'][17,7]=1
    Rxn['Law'][18,0]=2
    Rxn['Law'][18,7]=1
    Rxn['Law'][19,0]=2
    Rxn['Law'][19,8]=1
    Rxn['Law'][20,1]=2
    Rxn['Law'][20,9]=1
    Rxn['Law'][21,8]=1
    Rxn['Law'][22,9]=1
    Rxn['Law'][23,10]=1
    Rxn['Law'][24,10]=1
    Rxn['Law'][25,1]=1

    # Chromatin reactions
    Rxn['Law'][26,2]=1 # A_00->A_c
    Rxn['Law'][27,3]=1
    Rxn['Law'][28,4]=1
    Rxn['Law'][29,5]=1
    Rxn['Law'][30,6]=1 # A_c->A_00

    Rxn['Law'][31,7]=1 # B_00->B_c
    Rxn['Law'][32,8]=1
    Rxn['Law'][33,9]=1
    Rxn['Law'][34,10]=1
    Rxn['Law'][35,11]=1 # B_c->B_00

    # Reaction Stoichiometry, change in species resulting from reaction
    Rxn['Stoich'][0,0] = 1
    Rxn['Stoich'][1,0] = 1
    Rxn['Stoich'][2,0] = 1
    Rxn['Stoich'][3,0] = 1
    Rxn['Stoich'][4,2] = -1
    Rxn['Stoich'][4,0] = -2
    Rxn['Stoich'][4,3] = 1
    Rxn['Stoich'][5,2] = -1
    Rxn['Stoich'][5,1] = -2
    Rxn['Stoich'][5,4] = 1
    Rxn['Stoich'][6,3] = -1
    Rxn['Stoich'][6,1] = -2
    Rxn['Stoich'][6,5] = 1
    Rxn['Stoich'][7,4] = -1
    Rxn['Stoich'][7,0] = -2
    Rxn['Stoich'][7,5] = 1
    Rxn['Stoich'][8,3] = -1
    Rxn['Stoich'][8,0] = 2
    Rxn['Stoich'][8,2] = 1
    Rxn['Stoich'][9,4] = -1
    Rxn['Stoich'][9,1] = 2
    Rxn['Stoich'][9,2] = 1
    Rxn['Stoich'][10,5] = -1
    Rxn['Stoich'][10,0] = 2
    Rxn['Stoich'][10,4] = 1
    Rxn['Stoich'][11,5] = -1
    Rxn['Stoich'][11,1] = 2
    Rxn['Stoich'][11,3] = 1
    Rxn['Stoich'][12,0] = -1

    Rxn['Stoich'][13,1] = 1
    Rxn['Stoich'][14,1] = 1
    Rxn['Stoich'][15,1] = 1
    Rxn['Stoich'][16,1] = 1
    Rxn['Stoich'][17,7] = -1
    Rxn['Stoich'][17,1] = -2
    Rxn['Stoich'][17,8] = 1
    Rxn['Stoich'][18,7] = -1
    Rxn['Stoich'][18,0] = -2
    Rxn['Stoich'][18,9] = 1
    Rxn['Stoich'][19,8] = -1
    Rxn['Stoich'][19,0] = -2
    Rxn['Stoich'][19,10] = 1
    Rxn['Stoich'][20,9] = -1
    Rxn['Stoich'][20,1] = -2
    Rxn['Stoich'][20,10] = 1
    Rxn['Stoich'][21,8] = -1
    Rxn['Stoich'][21,1] = 2
    Rxn['Stoich'][21,7] = 1
    Rxn['Stoich'][22,9] = -1
    Rxn['Stoich'][22,0] = 2
    Rxn['Stoich'][22,7] = 1
    Rxn['Stoich'][23,10] = -1
    Rxn['Stoich'][23,1] = 2
    Rxn['Stoich'][23,9] = 1
    Rxn['Stoich'][24,10] = -1
    Rxn['Stoich'][24,0] = 2
    Rxn['Stoich'][24,8] = 1
    Rxn['Stoich'][25,1] = -1

    Rxn['Stoich'][26,2] = -1
    Rxn['Stoich'][26,6] = 1
    Rxn['Stoich'][27,3] = -1
    Rxn['Stoich'][27,6] = 1
    Rxn['Stoich'][27,0] = 2
    Rxn['Stoich'][28,4] = -1
    Rxn['Stoich'][28,6] = 1
    Rxn['Stoich'][28,1] = 2
    Rxn['Stoich'][29,5] = -1
    Rxn['Stoich'][29,6] = 1
    Rxn['Stoich'][29,0] = 2
    Rxn['Stoich'][29,1] = 2
    Rxn['Stoich'][30,2] = 1
    Rxn['Stoich'][30,6] = -1

    Rxn['Stoich'][31,7] = -1
    Rxn['Stoich'][31,11] = 1
    Rxn['Stoich'][32,8] = -1
    Rxn['Stoich'][32,11] = 1
    Rxn['Stoich'][32,1] = 2
    Rxn['Stoich'][33,9] = -1
    Rxn['Stoich'][33,11] = 1
    Rxn['Stoich'][33,0] = 2
    Rxn['Stoich'][34,10] = -1
    Rxn['Stoich'][34,11] = 1
    Rxn['Stoich'][34,0] = 2
    Rxn['Stoich'][34,1] = 2
    Rxn['Stoich'][35,7] = 1
    Rxn['Stoich'][35,11] = -1
    
    return Rxn, A, B, NumStates, NumSpec, NumRxn, GeneA_States, GeneB_States


def Determine_StatesDict(Dimensions, GeneA_States, GeneB_States):
    # original stateslist creation, returns ordered numpy array
    States_dict = {}

    for i in range(Dimensions[0]):
        for j in range(Dimensions[1]):
            for k in range(Dimensions[2]):
                for l in range(Dimensions[3]):
                    Cur = (i, j, *GeneA_States[k], *GeneB_States[l])
                    CurInd = np.ravel_multi_index((i, j, k, l), Dimensions, order='F')
                    States_dict[Cur] = CurInd
    return States_dict


def Calc_RateMatrix( Rxn, StatesDict, NumStates, NumRxn):
    StatesKeys = set(StatesDict.keys())
    MaxNumInteractions = NumStates * NumRxn
    CurInd_vals = np.zeros((MaxNumInteractions))
    DestInd_vals = np.zeros((MaxNumInteractions))
    RateMatrix_vals = np.zeros((MaxNumInteractions))

    n=0
    for state in StatesKeys:
        Cur = np.array(state, dtype=int)
        CurInd = StatesDict[state]
        for m in range(NumRxn):
            TestDest = tuple(Cur + Rxn['Stoich'][m,:])
            if TestDest in StatesKeys:
                n=n+1
                DestInd = StatesDict[TestDest]
                CurInd_vals, DestInd_vals, RateMatrix_vals = Update_RateMatrix(CurInd_vals, DestInd_vals, RateMatrix_vals, Cur, CurInd, DestInd, Rxn['Parameters'], Rxn['Law'], m, n)

    RateMatrix = coo_matrix((RateMatrix_vals, (DestInd_vals, CurInd_vals)), shape=(NumStates, NumStates)).tolil()
    RateMatrix.setdiag((RateMatrix.diagonal() - RateMatrix.sum(axis=0)).A[0])
    RateMatrix = RateMatrix.tocsc()

    return RateMatrix

def main(inputs):
    Rxn, A, B, NumStates, NumSpec, NumRxn, GeneA_States, GeneB_States = MISAChromatin_Rxn(inputs)
    Dimensions = [len(A), len(B), len(GeneA_States), len(GeneB_States)]
    StatesDict = Determine_StatesDict(Dimensions, GeneA_States, GeneB_States)
    RateMatrix = Calc_RateMatrix( Rxn, StatesDict, NumStates, NumRxn)

    return RateMatrix, Dimensions, StatesDict
